In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/media/workstation/Storage/GoogleProject/DeepLearningAlphaC.txt', scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open("TASK")
worksheet_list = spreadsheet.worksheets()
Accepted = spreadsheet.worksheet("Accepted").get_all_records()

In [7]:
def make_str(x):
    return '"{}"'.format(x)

def create_data_exclusion_list(RefInstrument):
    x = len(RefInstrument.split(","))
    x2 = ",".join([str(j) for j in range(x+1)])
    x3 = make_str(x2)
    return x3

def create_num_instruments(RefInstrument):
    x = len(RefInstrument.split(","))
    return str(x)

def get_ref_instrument(Asset_B, Asset_S):
    RefInstrument = Asset_B if Asset_B != "-" else Asset_S
    return RefInstrument.replace(" ", "")

def load_cs_file(template_path):
    fo = open(template_path, "r+")
    line = fo.readlines()
    return line

def var_replace(x):
    return "= {};".format(x)
    
def load_and_replace_line(**d):
    # Load template
    template_dir = "/home/workstation/Desktop/cxt-strategies/Development/Jun Yitt"
    template_filename = "@@_0000_MongoDLTemplate.cs"
    template_path = os.path.join(template_dir, template_filename)
    line = load_cs_file(template_path)
    
    # Replace default parameters accordingly
    for i, z in enumerate(line):
        if i < 250:
            # Direct replace
            if z.find("_0000_MongoDLTemplate") != -1:
                line[i] = z.replace("_0000_MongoDLTemplate", d.get("Name"))
            
            for key, val in d.items():
                if key != "Name":
                    if z.find("\t{}".format(key)) != -1:
                        q1 = z.find("=")
                        q2 = z.find(";") + 1
                        to_replace = z[q1:q2]
                        replace_with = var_replace(val)
                        line[i] = z.replace(to_replace, replace_with)
    
    return line

def write_line_to_file(output_folder, StratName, line):
    dir_folder = output_folder + "/{}.cs".format(StratName)
    f = open(dir_folder, 'w')
    f.writelines(line)
    f.close()


    

In [8]:
output_folder = "20190706_Output_MongoDL_Batch_1"
try:
    os.mkdir(output_folder)
except:
    print("folder exist!")

folder exist!


In [9]:
MongoDBIPAddress = make_str("192.168.1.110")
Port = make_str("27017")
SignalDB = make_str("MLProduction")
SignalCollectionName = make_str("SignalOneBB2")

BarsLag = '0'
BarsLagTolerance = '1'

# Name = "_1286..."

# StrategyCode = '"Z_C007"'
# RefInstrument = '"AUDUSD,NZDUSD,XAUUSD"'
# ConsecutiveSignalBars = '5'
# OrderType = '2'

# IsSixtyMinuteStrategy = 'false'
# DataExclusionList = '"0,1,2,3"'
# NumInstruments = create_num_instruments(RefInstrument)

In [10]:
for d in Accepted:
#     if d.get("Backtest Status") != "Abandon":
        TimeFrame = int(d.get("TimeFrame").split("_")[1])

        #Direct replace
        Name = d.get("StratName")

        # Replace value
        MongoDBIPAddress = MongoDBIPAddress
        Port = Port
        SignalDB = SignalDB
        SignalCollectionName = SignalCollectionName

        StrategyCode = make_str(d.get("Code"))
        RefInstrument = make_str(get_ref_instrument(d.get("Asset_B"), d.get("Asset_S")))
        ConsecutiveSignalBars = str(d.get("HoldingPeriod"))
        OrderType = str(d.get("OrderType"))

        BarsLag = BarsLag
        BarsLagTolerance = BarsLagTolerance

        if TimeFrame == 60:
            IsSixtyMinuteStrategy = 'true'
        else:
            IsSixtyMinuteStrategy = 'false'

        DataExclusionList = create_data_exclusion_list(RefInstrument) 
        NumInstruments = create_num_instruments(RefInstrument)
        
        # Load Template, and Replace default parameters
        line = load_and_replace_line(Name = Name, MongoDBIPAddress = MongoDBIPAddress, Port = Port, SignalDB = SignalDB, SignalCollectionName = SignalCollectionName, 
                     StrategyCode = StrategyCode, RefInstrument = RefInstrument, ConsecutiveSignalBars = ConsecutiveSignalBars, 
                     OrderType = OrderType, BarsLag = BarsLag, BarsLagTolerance = BarsLagTolerance, IsSixtyMinuteStrategy = IsSixtyMinuteStrategy,
                     DataExclusionList = DataExclusionList, NumInstruments = NumInstruments)
        
        # Write lines to .cs file
        write_line_to_file(output_folder, Name, line)

    